In [1]:
import sys
sys.path.insert(0, '../')
from utils.GraphAnalytics import GraphAnalytics
from models.Node2Vec import GraphNode2Vec

In [2]:
g = GraphAnalytics()
g.load_graph('../data/_tmp')

In [3]:
mod = GraphNode2Vec()

hello


In [4]:
mod.fit(g.as_undigraph)

In [5]:
import pandas as pd
pd.DataFrame(mod.embeddings).transpose()

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,-0.175778,0.552505,-0.462174,-0.472928,-0.563834,0.106522,0.686009,0.283666,0.062580,-0.155229,...,-1.073352,-0.277859,-0.159773,0.204890,-0.123218,0.002878,0.384899,-0.070671,0.159951,0.132984
21966,-0.005740,0.455905,-0.422919,-0.282476,-0.471488,-0.000416,0.593768,0.213291,0.044341,-0.200531,...,-0.845800,-0.321243,-0.007249,0.139174,-0.078471,-0.041425,0.340471,-0.021613,0.123464,-0.000010
3,-0.208993,0.564582,-0.450227,-0.538043,-0.566572,0.117799,0.719168,0.375214,0.078885,-0.021688,...,-1.082621,-0.311054,-0.210596,0.272664,-0.190728,0.046915,0.355629,-0.069532,0.116721,0.168612
2280,-0.065933,0.489389,-0.440764,-0.275606,-0.487456,0.058844,0.575128,0.220699,0.049661,-0.223927,...,-0.877968,-0.283664,-0.022893,0.126415,-0.058381,0.006235,0.345298,-0.025261,0.128233,0.031922
35934,-0.107524,0.553454,-0.484260,-0.309959,-0.539312,0.094202,0.625832,0.233543,0.054615,-0.253149,...,-0.991080,-0.283012,-0.042918,0.140375,-0.064141,0.021906,0.376853,-0.032962,0.150250,0.053295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,-1.011953,0.511764,-0.361351,-0.702189,-0.334235,0.764528,-0.020222,0.048567,0.008477,-0.147001,...,-0.501456,-0.016284,-0.563843,0.442280,-0.137210,0.014506,-0.035700,-0.187167,-0.093699,0.072000
2035,-0.713970,0.351215,-0.275599,-0.560294,-0.239165,0.514875,-0.004085,0.058394,-0.007865,-0.041067,...,-0.367650,-0.038459,-0.463057,0.372173,-0.147325,-0.047611,0.005995,-0.133723,-0.072272,0.063990
1533,-0.820207,0.397233,-0.307477,-0.630720,-0.267912,0.589007,-0.003661,0.076919,-0.005963,-0.022907,...,-0.424817,-0.037849,-0.525122,0.416717,-0.168459,-0.012202,0.004214,-0.159202,-0.088890,0.087548
555,-0.739617,0.676900,-0.270708,-0.005245,-0.174363,0.632614,0.217727,0.211243,0.121468,-0.180224,...,-0.258301,-0.107040,0.133693,0.058288,-0.107996,0.436426,-0.112357,-0.192925,-0.228164,0.099597
